<a href="https://colab.research.google.com/github/rwong-current/colab/blob/main/utils/gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install required libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# Authenticate the user
from google.colab import auth
auth.authenticate_user()

# Import necessary libraries
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.auth

# Create the Drive API service
class drive:
  def __init__(self):
    creds, _ = google.auth.default()
    self.service = build('drive', 'v3', credentials=creds)

  def get_folder_id(self, folder_name, parent_folder_id=None):
      """Get the folder ID of an existing folder, or create it if it doesn't exist."""
      query = f"name = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
      if parent_folder_id:
          query += f" and '{parent_folder_id}' in parents"

      results = self.service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
      items = results.get('files', [])

      if items:
          # Folder exists
          return items[0]['id']
      else:
          # Folder does not exist, create it
          file_metadata = {
              'name': folder_name,
              'mimeType': 'application/vnd.google-apps.folder'
          }
          if parent_folder_id:
              file_metadata['parents'] = [parent_folder_id]

          try:
              folder = self.service.files().create(body=file_metadata, fields='id').execute()
              return folder.get('id')
          except HttpError as error:
              print(f"An error occurred: {error}")
              return None

  def create_folder_path(self, path):
      """Create a folder path in Google Drive."""
      folder_names = path.strip('/').split('/')
      parent_folder_id = None

      for folder_name in folder_names:
          folder_id = self.get_folder_id(folder_name, parent_folder_id)
          if folder_id:
              parent_folder_id = folder_id
          else:
              print(f"Failed to create or find folder: {folder_name}")
              return None

      if parent_folder_id:
          print(f"Folder path '{path}' created with final folder ID: {parent_folder_id}")
      else:
          print(f"Failed to create folder path '{path}'")

      return parent_folder_id


Folder path 'Colab Notebooks/test_folder' created with final folder ID: 146DAKrf9d_aqKsz-fWGgTFDhamfL0lPa
